# Redes Convolucionales
Las redes convolucionales, o CNN (Convolutional Neural Network), son un tipo especializado de redes neuronales que han sido aplicado con mucho éxito en problemas en cuales los datos tienen forma de grillas, como son las imagenes. Se las conocen como redes convolucionales ya que aplican una operación matemática conocida como convolución. La convolución no es más que un operador móvil que se aplica repetidamente sobre los datos de entrada. Este operador está definido por una matríz pequeña, generalmente llamada kernel, que se aplica repetidamente sobre la imagen. Por ejemplo, imaginemos un kernel de 2 X 2.

$$K=\left[\begin{array}{cc}
k_{1,1} & k_{1,2}\\
k_{2,1} & k_{2,2}
\end{array}\right]$$

y una imagen de n X m:

$$I=\left[\begin{array}{cc}
i_{1,1} & i_{1,2} & ... & i_{1, m}\\
i_{2,1} & i_{2,2} & ... & i_{2, m}\\
... & ... & ... & ... \\
i_{n,1} & i_{n,2} & ... & i_{n, m}\\
\end{array}\right]$$

el resultado de aplicar la convolución sería:

$$C=\left[\begin{array}{cc}
c_{1,1} & c_{1,2} & ... & c_{1, m-1}\\
c_{2,1} & c_{2,2} & ... & c_{2, m-1}\\
... & ... & ... & ... \\
c_{n-1,1} & c_{n-1,2} & ... & c_{n-1, m-1}\\
\end{array}\right]$$

donde:

$$c_{i, j} = i_{i, j} * k_{1,1} + i_{i, j+1} * k_{1,2} + i_{i+1, j} * k_{2,1} + i_{i+1, j+2} * k_{2,2}$$

La operación de convolución ha sido usada con mucho exito en procesamiento de imagenes para detección de bordes, mejoramiento de imagenes, aplicación de blur, etc. Por ejemplo, Kirsch[1] propuso en 1971 una técnica que permite detectar estrucuras en las imagenes. Para esto, utiliza distintas [matrices de convolución](https://en.wikipedia.org/wiki/Kirsch_operator). Para ilustrar el la convolución utilizaremos $g^{(1)}$.

$$g^{(1)}=\left[\begin{array}{cc}
5 & 5 & 5 \\
-3 & 0 & -3 \\
-3 & -3 & -3
\end{array}\right]$$

[1] Kirsch, R. (1971). "[Computer determination of the constituent structure of biological images](https://www.sciencedirect.com/science/article/pii/0010480971900346)". Computers and Biomedical Research. 4: 315–328. doi:10.1016/0010-4809(71)90034-6.

In [ ]:
#Cargamos bibliotecas y demás
!pip install tqdm
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from keras.datasets import mnist
from sklearn.metrics import accuracy_score
from keras.layers import Input, Dense, Conv2D, Flatten
from keras.models import Model 
from keras.utils import to_categorical
from keras import backend as K
from scipy.misc import imread
from tqdm import tqdm
import os.path
while not os.path.exists('movediza.jpg'):
    #Si no está el archivo hay que subirlo. Solo para Google Colab!!
    from google.colab import files
    uploaded = files.upload()
    for fn in uploaded.keys():
        print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

In [ ]:
#Cargo una imagen.
movediza = imread('movediza.jpg')
print(movediza.shape)
plt.imshow(movediza)
plt.show()

In [ ]:
#Cargo la matriz
kirsch_matrix = [[5, 5, 5], 
                [-3, 0, -3],
                [-3, -3, -3]]
kirsch_matrix = np.asarray(kirsch_matrix, dtype=np.float32) 
#Inicializo el placeholder como una matriz con garbage
movediza_k1 = np.empty((movediza.shape[0]-2, movediza.shape[1]-2, 3))
#Convolucion por fila, por columna, 
for i in tqdm(range(0, movediza.shape[0]-2)):
    for j in range(0, movediza.shape[1]-2):
        #Trato los canales de forma independiente
        for c in range(3): 
            # El operador * multiplica la matriz elemento a elemento y luego la
            # reduzco con una suma
            movediza_k1[i, j, c] = np.sum(movediza[i:i+3, j:j+3, c] * kirsch_matrix)
# Normalizo la imagen para poder mostrarla
movediza_k1 = (movediza_k1 - np.min(movediza_k1))/(np.max(movediza_k1) - np.min(movediza_k1))
plt.imshow(movediza_k1)
plt.show()

## ¡Ahora con keras!

In [ ]:
# De entrada tengo una matriz de 3 dimensiones, las primeras dos
# son de tamaño variable
i = Input(shape=(None, None, 3)) 
# Agrego una capa convolucional de 2 dimensiones
d = Conv2D(3, (3,3), activation='linear', use_bias=False)(i)
# Creo el modelo
kirsch = Model(inputs=i, outputs=d)
kirsch.summary()
# Compilo el modelo. No importa el loss ni optimizer.
kirsch.compile(loss='categorical_crossentropy', optimizer='sgd')
print('Forma de los parámetros de la convolución: {}'.format(kirsch.layers[1].kernel.shape))
# Cargo la matrix en un kernel apropiado
kirsch_kernel = np.zeros((3, 3, 3, 3))
for i in range(3):
    kirsch_kernel[:, :, i, i] = kirsch_matrix
# Utilizo el backend para cargar el kernel construido.
K.set_value(kirsch.layers[1].kernel, kirsch_kernel)
# Aplico la convolución.
movediza_k2 = kirsch.predict(np.asarray([movediza]))[0]
# Normalizo la imagen para poder mostrarla
movediza_k2 = (movediza_k2 - np.min(movediza_k2))/(np.max(movediza_k2) - np.min(movediza_k2))
plt.imshow(movediza_k2)
plt.show()

## Explicación del código
Como primera etapa, creo la estructura de la CNN. En particular es de interes la creación de la capa convolucional 2D:

```d = Conv2D(3, (3,3), activation='linear', use_bias=False)(i)```

El primer parámetro indica cuantos canales tiene la imagen de salida. En este caso, como el objetivo es una imagen RGB la cantidad de canales es 3. El segundo parámetro indica que función de activación se aplica sobre la convolución. En este caso, se aplica la función lineal/identidad $f(x)=x$. Finalmente, indicamos que no se utilizará un parametro de bías, equivalente al $b$ en la regresión lineal o logística. 

Como segunda etapa, compilamos el modelo. Se especifican función de perdida y optimizador porque son obligatorios, pero solo son interesa compilar por lo que estos parámetros pueden ser cualquiera.

```kirsch = Model(inputs=i, outputs=d)
kirsch.summary()
kirsch.compile(loss='categorical_crossentropy', optimizer='sgd')```

A diferencia de la matriz original de 3x3, la matriz kernel de la capa convolucional es de 3x3x3x3. Esto se debe a que la operación de convolución en este caso es más generico. Las primeras dos dimensiones efectivamente se corresponde con las dimensiones de la convolución. La tercera dimensión se corresponde con la cantidad de canales de la imagen, ya que la convolución en este caso tambíen considera los canales. Finalmente, la cuarta dimensión se corresponde con la cantidad de canales de salida.

```kirsch_kernel = np.zeros((3, 3, 3, 3))
for i in range(3):
    kirsch_kernel[:, :, i, i] = kirsch_matrix
K.set_value(kirsch.layers[1].kernel, kirsch_kernel)```

Como queremos que la convolución se aplice por cada canal, lo único que hay que hace es asignarle la matriz de convolución al canal correspondiente, dejando todos los otros valores en cero. Como resultado, se ignoran los valores en los otros canales, ya que se multiplican por cero (elemento absorbente para la multiplicación) y luego se suman (elemento neutro en la suma).

## Intuiciones
Como se mostró en el ejemplo anterior, los filtros/kernels convolucionales pueden aplicarse para detectar ciertas características de las imagenes como son los bordes. Además, estos kernels tiene relativamente pocos parámetros. En este caso, la cantidad de parámetros eran 81 (3x3x3x3). Si se aplicara una red densa sobre la imagen de entrada de 640x480x3 y se espera una salida de aproximadamente el mismo tamaño se requeririan 8.4924656e11 ((640x480x3)^2) parámetros.

## Ejercicio
1. Cargue el dataset MNIST y transformelo de forma que pueda ser utilizado por una red convolucional. Recuerde que las intancias de X tiene la forma (instancias, 28, 28), pero las redes convolucionales esperan algo de la forma (instancias, ancho, alto, canales). Además, recuerde que Y debe ser tratado de forma categórica.
1. Cree una red neuronal convolucional para clasificar los dígitos. 
    1. Utilice 4 o 5 convoluciones.
    1. Puede incrementar el número de filtros en las distintas convoluciones, por ejemplo 3, 5, 10, 15.
    1. Utilice kernels de 3x3 hasta 7x7. 
    1. Las capaz de tipo ```Flatten()``` pueden ser utilizadas para aplanar la salida de la última convolución.
    1. Aunque admiten cualquier función de activación, generalmente se utiliza la relu.
    1. Utilice una o dos capaz densas para realizar la clasificación.
    1. El optimizador rmsprop puede tiende a llegar más rápido a un mínimo.
1. Compare la cantidad de parámetros de su red con los parámetros de las redes construidas en la lección anterior. Comparte el accuracy de estas redes.

In [ ]:
#Carga de dataset

In [ ]:
#Modelo

## Ejercicio
Visualize las distintas salidas de las capaz convolucionales para una instancia dada. Para obtener el valor de una capa intermedia, es necesario compilar una función en el backend. Puede utilizar el siguiente código para hacerlo:

``` m = K.function(inputs=[model.layers[0].input, K.learning_phase()], outputs=[model.layers[n_layer].output])
pred = m([x_test[idx:idx+1, :, :, :], 0.])[0]```

In [ ]:
idx = 1
print('Número de ejemplo')
ax = plt.subplot(111)
ax.matshow(x_test[idx, :, :, 0], cmap='gray')
ax.set_yticklabels([])
ax.set_xticklabels([])
plt.show()
#Ponga su código para ver las capas intermedias de su modelo aquí

### Pooling
Pooling es una operación que reduce la salida de una red en una posición utilizando alguna función estádistica de la misma. Por ejemplo, *Max pooling* reemplaza una posición y sus vecinos por el máximo valor en el vecindario. Keras proporciona diversas capas de pooling.
* `MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)`: Obtine el maximo de un vecindario de tamaño `pool_size`, moviendose en la imagen utilizando los desplazamientos indicados en `strides` (si es `Nono` `strides=pool_size`).
* `AveragePooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)`: Obtine el maximo de un vecindario de tamaño `pool_size`, moviendose en la imagen utilizando los desplazamientos indicados en `strides` (si es `Nono` `strides=pool_size`).
* `GlobalMaxPooling2D(data_format=None)`: Calcula el máximo por canal, transformando una entrada de forma `(batch_size, rows, cols, channels)` a una salida de forma `(batch_size, channels)`.
* `GlobalAveragePooling2D(data_format=None)`: Calcula el promedio por canal, transformando una entrada de forma `(batch_size, rows, cols, channels)` a una salida de forma `(batch_size, channels)`.

## Overfitting
Overfitting es un fenómeno que ocurre cuando el modelo entrenado se ajusta tan bien a los datos de entrenamiento, que falla al generalizar para nuevos datos. Este es un problema muy común en las redes neuronales ya que estas tienen mucha capacidad para adaptarse a los datos de entrenamiento. En general, cuantos más parámetros entrenables tenga una red neuronal es más probable que sufra algún tipo de overfitting.

### Ejercicio
Adapte la red neuronal que utilizó para el problema MNIST al problema [CIFAR-100](https://www.cs.toronto.edu/~kriz/cifar.html). Este conjunto de datos tiene 100 categorías con 600 imagenes RGB de 32x32 para cada una. 500 imagenes por categorías están asignadas al entrenamiento y 100 a la validación. Puede cargar el conjunto de datos de la siguiente manera:

```from keras.datasets import cifar100
(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')```

Una métrica común para problemas con tantas clases es el top-k accuracy. En esta métrica se considera acierto si la clase real está entre las k clases más probables. En partícular, se suele usar k=5. Esta métrica se encuentra implementada en keras, y la puede utilizar importandola desde:

`from keras.metrics import top_k_categorical_accuracy`

Utilizando el retorno del método `fit(...)` del modelo, grafique `'loss'` vs `'val_loss'`, `'top_k_categorical_accuracy'` vs `'val_categorical_accuracy'`, y `'top_k_categorical_accuracy'` vs `'val_top_k_categorical_accuracy'` con respecto a los epochs.

In [ ]:
#Código aquí

## Tecnicas para evitar overfitting
Existen varias técnicas para reducir el overfitting.

### Regularización
Regulirización consiste en modificar la función de perdidad un componente más que penelice cuando se utilicen pesos grandes. Considerando una función de perdidad $J(y_{true},y_{pred})$, la función de perdida con regularización sería:

$$\hat{J}(y_{true},y_{pred},W) = J(y_{true},y_{pred})+C*REG(W)$$

Donde $C$ es el peso de la regularización y $REG(W)$ es una función de perdida asociada a los pesos. En particular existen dos regularizaciones que se suelen utilizar utilizando las normas L1 y L2.

$$\left|W\right|_1=\sum abs(W_i)$$

$$\left|W\right|_2=\sum (W_i^2)$$

Keras permite definir la regularización a nivel capa. En general las capas que soportan kernel (lo que notamos como $w$) y bias, tienen los parámetros `kernel_regularizer=None`, `bias_regularizer=None` en su constructor. Si se utilizan los strings `'l1'`, `'l2'`, o `'l1l2'` se utiliza regularización L1, L2, o L1+L2 con $C=0.1$. En caso de querer cambiar los parámetros se debe importar la clase `keras.regularizers.L1L2` e inicializarla con los parámetros `L1L2(l1=C1, l2=C2)`.

### Noise: Dropout, GausianDropout, GausianNoise
Otra solución es agregar capas que generen ruido en la red durante el entrenamiento. 
* `Dropout(rate)`: Durante el entrenamiento, cada elemento de la entrada tiene una `rate` posibilidad de ser llevado a cero en la salidad.
* `GausianDropout(rate)`: Durante el entrenamiento, cada elemento entrada será multiplicado por un ruido con distribución normal de media 1 y desviación estandard  `sqrt(rate / (1 - rate))`.
* `GaussianNoise(stddev)`: Durante el entrenamiento, cada elemento entrada se le sumará un ruido con distribución normal de media 0 y desviación estandard  `stddev`.

### Data Augmentation
Data Augmentation es una técnica que consiste en generar nuevos datos a partir de los datos de entrenamiento a partir de transformaciones que no modifiquen la semantica de los mismo. En el caso de las imagenes, algunas de las transformacións que se suelen aplicar son rotaciones, transalaciones, modificaciones en los colores, entre otros.
En este caso, keras provee un generador que modifica imagenes bases. Los generadores son iteradores que retornan tuplas `(x_train, y_train)` con batchs de entrenamiento. Un ejemplo de como utilizar el generador provisto por Keras:

```from keras.preprocessing.image import ImageDataGenerator
img = ImageDataGenerator(params...)
model.fit_generator(img.flow(x_train, y_train, batch_size=100), steps_per_epoch=500, epochs=50, ....)```

A diferencia del método `fit`, el método `fit_generator` recibe un iterador que retorna las imagenes transformadas. Se le debe expecificar la cantidad de pasos (batchs que procesará) por epochs. Mientras que el tamaño del batch está definido por la cantidad de elementos que retorna el iterador en cada iteración. 

### EarlyStopping
Esta estrategia se basa en dejar de entrenar cuando una métrica en particular deja de mejorar. En general, se establece una métrica sobre el conjunto de validación. Keras lo implementa como un callback que es llamado por los métodos de entrenamiento al final de cada epoch.

### ReduceLROnPlateau y LearningRateScheduler
No son técnicas específica para reducir el overfitting, pero facilitan el entrenamiento ya que permiten reducir el *learning rate* a medida que pasa el tiempo. En general, un *learning rate* alto es bueno para empezar el entrenamiento ya que permite encontrar zonas con errores bajos rápidamente, pero es malo para aproximar el mínimo. Mientras que uno bajo es malo para empezar el entrenamiento, pero bueno para encontrar el mínimo.

## Trabajo práctico
Convinando las técnicas descriptas anteriormente, implemente una CNN para el problema CIFAR-100. Objetivos en el conjunto de validación:
* accuracy > 0.3.
* top-5 accuracy > 0.6.

**NOTA:** No es necesario que su solución utilice todas las técnicas descriptas.

### Opcional
Intente implementar otra solución utilizando módulos fire que se describen en la arquitectura [SqueezeNet](https://arxiv.org/abs/1602.07360).

In [ ]:
#TP aquí

In [ ]:
#Opcional aquí